In [9]:
import numpy as np

In [21]:
moves = ["G1","G2","G3"]

def extract_coords_2d(cmd):
    words = cmd.split(" ")
    x,y = None, None
    for w in words:
        if words[0] in moves:
            if "X" in w:
                x = float(w[1:])
            if "Y" in w:
                y = float(w[1:])
    return (x,y)

In [31]:
f = open("gcode/tube_path_explicit.pgm","r")
lines = f.read().splitlines()
f.close()

mvt_lines = []
commands = []
coords = []
for l in lines:
    cmd = l.split(" ")[0]
    if cmd not in moves:
        continue
    commands.append(cmd)
    mvt_lines.append(l)
    x,y = extract_coords_2d(l)
    if x == None and y != None:
        x = coords[-1][0]
    if x != None and y == None:
        y = coords[-1][1]
    coords.append([x,y])

In [47]:
output = ["G91","ENABLE X Y","G16 X Y Z","G17"]
for k in range(1,len(mvt_lines)):
    l = mvt_lines[k]
    cmd = commands[k]
    dx = coords[k][0] - coords[k-1][0]
    dy = coords[k][1] - coords[k-1][1]
    if cmd == "G1":
        line = "G1 X{0:.4f} Y{1:.4f} F40".format(dx,dy)
    if cmd == "G2" or cmd == "G3":
        i,j = None, None
        for w in l.split(" "):
            if "I" in w:
                i = w
            if "J" in w:
                j = w
        line = "{0} X{1:.4f} Y{2:.4f} {3} {4}".format(cmd,dx,dy,i,j)
    output.append(line)
output.append("DISABLE X Y")

In [51]:
fout = open("gcode/tube_path_relative.pgm","w")
for l in output:
    fout.write(l+"\n")
fout.close()

In [30]:
k = [i for i in range(1,5)]
k

[1, 2, 3, 4]